In [1]:
# Ensemble training (1 year of testing, 2 years of validation, the rest is training)
    # Learned weights on validation data (Constrained optimization)
    # Meta-learning using an other ML model

# Models used:
# - XGBoost
# - Neural Network
# - Logistic Regression
# - LSTM


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os


In [5]:
# Set up paths and load data
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
print(parent_dir)

interpolated_dir = os.path.join(parent_dir, "dataset_interpolated_fixed")
features = ["game_completed", "relative_strength", "score_difference", "type.id", "home_has_possession", "end.down", "end.yardsToEndzone", "end.distance", "field_position_shift", "home_timeouts_left", "away_timeouts_left"]
# Import necessary modules
from sklearn.metrics import brier_score_loss, accuracy_score, log_loss
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize
import pickle

/Users/aly/Documents/University_of_Waterloo/Winter 2025/Research/code/NFL/ML


In [6]:
# Load data for ensemble training
import process_data
# training_data = process_data.load_data(interpolated_dir, 
#                                        years = [2016, 2017, 2018, 2019, 2020, 2021], 
#                                        history_length = 0, 
#                                        features = features, 
#                                        label_feature = "home_win")

training_data = process_data.load_data(interpolated_dir, 
                                       years = [2021], 
                                       history_length = 0, 
                                       features = features, 
                                       label_feature = "home_win")


ensemble_data = process_data.load_data(interpolated_dir, 
                                         years = [2023], 
                                         history_length = 0, 
                                         features = features, 
                                         label_feature = "home_win",
                                         train = True
                                         )

test_data = process_data.load_data(interpolated_dir, 
                                   years = [2024],
                                   history_length = 0, 
                                   features = features, 
                                   label_feature = "home_win",
                                   train = False
                                   )

# training_data_seq = process_data.load_data(interpolated_dir, 
#                                        years = [2016, 2017, 2018, 2019, 2020], 
#                                        history_length = 4, 
#                                        features = features, 
#                                        label_feature = "home_win",
#                                        train = True
#                                        )

training_data_seq = process_data.load_data(interpolated_dir, 
                                       years = [2020], 
                                       history_length = 4, 
                                       features = features, 
                                       label_feature = "home_win",
                                       train = True
                                       )

ensemble_data_seq = process_data.load_data(interpolated_dir, 
                                         years = [2023], 
                                         history_length = 4, 
                                         features = features, 
                                         label_feature = "home_win",
                                         train = True
                                         )

test_data_seq = process_data.load_data(interpolated_dir, 
                                   years = [2024],
                                   history_length = 4, 
                                   features = features, 
                                   label_feature = "home_win",
                                   train = False
                                   )


Loading data for 2022
skipping  2022
Loading data for 2024
skipping  2024
Loading data for 2023
skipping  2023
Loading data for .DS_Store
Loading data for 2017
skipping  2017
Loading data for 2019
skipping  2019
Loading data for 2021
  Processing 272 CSV files in parallel with 8 workers...
  NaN found in file: /Users/aly/Documents/University_of_Waterloo/Winter 2025/Research/code/NFL/ML/dataset_interpolated_fixed/2021/game_401326405.csv
  NaN found in file: /Users/aly/Documents/University_of_Waterloo/Winter 2025/Research/code/NFL/ML/dataset_interpolated_fixed/2021/game_401326412.csv
  NaN found in file: /Users/aly/Documents/University_of_Waterloo/Winter 2025/Research/code/NFL/ML/dataset_interpolated_fixed/2021/game_401326412.csv
  Completed processing 2021
Loading data for 2020
skipping  2020
Loading data for 2018
skipping  2018
Loading data for 2016
skipping  2016
Loading data for 2022
skipping  2022
Loading data for 2024
skipping  2024
Loading data for 2023
  Processing 272 CSV files 

In [160]:
from sklearn.model_selection import train_test_split
import shap
class EnsemblePredictor:
    """
    Ensemble predictor class, one per timestep
    """
    def __init__(self, all_models, all_models_order, all_features, strategy='meta_model', meta_model=None):
        self.all_models = all_models
        self.all_models_order = all_models_order
        self.strategy = strategy
        self.all_features = all_features
        if self.strategy != 'meta_model' and self.strategy != 'weighted_average':
            raise ValueError("Invalid strategy")
        if meta_model is None and self.strategy == 'meta_model':
            raise ValueError("Meta model is required for meta_model strategy")
        self.meta_model = meta_model
        self.ensemble_weights = None # Will be a 1D array of shape (n_models,) once trained
 
    def train_ensemble(self, x_train, y_train, objective='brier'):
        """
        Train the ensemble for a single timestep using validation data.
        """
        print(f"Training ensemble for this timestep...")
        if self.strategy == 'weighted_average':
            self.optimize_ensemble_weights(x_train, y_train, objective)
        elif self.strategy == 'meta_model':
            self.train_meta_model(x_train, y_train)

    def optimize_ensemble_weights(self, x_train, y_train, objective='brier'):
        """
        Optimize ensemble weights for a single timestep using validation data.
        """
        print(f"Optimizing ensemble weights for this timestep...")

        n_models = x_train.shape[1]

        def objective_function(weights):
            weights = weights / np.sum(weights)  # Normalize weights
            ensemble_preds = np.dot(x_train, weights)

            if objective == 'brier':
                return brier_score_loss(y_train, ensemble_preds)
            elif objective == 'logloss':
                # Clip predictions to avoid log(0)
                ensemble_preds = np.clip(ensemble_preds, 1e-15, 1-1e-15)
                return log_loss(y_train, ensemble_preds)
            elif objective == 'accuracy':
                return -accuracy_score(y_train, ensemble_preds > 0.5)  # Negative for minimization

        # Constraints: weights sum to 1 and are non-negative
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
        bounds = [(0, 1) for _ in range(n_models)]

        # Initialize with equal weights
        initial_weights = np.ones(n_models) / n_models

        result = minimize(objective_function, initial_weights,
                          method='SLSQP', bounds=bounds, constraints=constraints)

        if result.success:
            self.ensemble_weights = result.x
            print(f"  Optimized weights: {dict(zip(self.all_models_order, result.x.round(4)))} (score: {result.fun:.6f})")
        else:
            print(f"  Optimization failed, using equal weights")
            self.ensemble_weights = initial_weights

        return self.ensemble_weights

    def train_meta_model(self, x_train, y_train):
        """
        Train a meta-model for a single timestep to predict based on base model outputs.
        """
        X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
        # Train meta-model: input=base_model_predictions, output=final_prediction
        self.meta_model.fit(X_train, y_train.reshape(-1, 1))

        # Test the meta-model's prediction capability
        meta_predictions = self.meta_model.predict_proba(X_val)[:, 1]
        meta_accuracy = accuracy_score(y_val, meta_predictions > 0.5)
        meta_brier = brier_score_loss(y_val, meta_predictions)

        print(f"  Meta-model trained on {len(X_train)} samples")
        print(f"    Validation Meta-model accuracy: {meta_accuracy:.4f}, Validation Brier score: {meta_brier:.4f}")

        return self.meta_model
    
    def predict(self, X):
        """
        Predict the probability of the positive class for a given input
        """
        # Generate predictions from each individual model
        # Convert X to a 3D array of shape (n_samples, n_history, n_features)
        predictions = [] # Will be a 2D array of shape (n_samples, n_models)
        for i, model_name in enumerate(self.all_models_order):
            model = self.all_models[model_name]
            print("Model name: ", model_name)
            if model_name == "lstm":
                print(X.shape)
                pred = model.predict_proba(X)
                print(pred)
                predictions.append(pred[:, 1])
            else:
                print(X.shape)
                x = np.array([X[i][-1] for i in range(X.shape[0])]) if len(X.shape) == 3 else np.array([X[-1]])
                print(x.shape)
                pred = model.predict_proba(x)
                print(pred)
                predictions.append(pred[:, 1]) # Will be a 1D array of shape (n_samples,) for each model
        predictions = np.array(predictions) # Will be a 2D array of shape (n_models, n_samples)
        predictions = predictions.T # Reshape to be a 2D array of shape (n_samples, n_models)
        if self.strategy == 'weighted_average':
            return np.dot(predictions, self.ensemble_weights)
        elif self.strategy == 'meta_model':
            return self.meta_model.predict_proba(predictions)[:, 1]
    
    def predict_proba(self, X):
        """
        Predict the probability of the positive class for a given input
        """
        pred = self.predict(X).flatten()
        return np.column_stack([1 - pred, pred])

    def predict_proba_single(self, X):
        preds = self.predict_proba(X)
        return preds[:, 1]

    def score(self, X, y):
        """
        Score the ensemble for a given input
        """
        y_pred = self.predict(X)
        y_pred_labels = (y_pred > 0.5).astype(int)
        return np.mean(y_pred_labels == y)
    
    def SHAP_analysis(self, X_test, X_train, plot = True):
        """
        Model interpretability with SHAP values
        """
        feature_names = self.all_features
        masker = shap.maskers.Independent(X_train[:10])
        explainer = shap.Explainer(self.predict_proba_single, masker, feature_names=self.all_features)
        shap_values = explainer(X_test)
        if plot:
            shap.plots.bar(shap_values)
        return shap_values


In [94]:
# Reload modules
modules_to_reload = [
    'models.xg_boost',
    'models.direct_prediction_network_lstm',
    'models.direct_prediction_network',
    'models.logistic_regression',
    'models.Model'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
from models.xg_boost import setup_xgboost_models
from models.direct_prediction_network_lstm import setup_direct_lstm_models
from models.direct_prediction_network import setup_direct_models
from models.logistic_regression import setup_logistic_regression_models


In [36]:
all_models = {}

In [97]:
all_models["lstm"] = setup_direct_lstm_models(training_data_seq, None, num_models=201)

Split training data: 450 train, 50 validation

Training direct prediction LSTM model for timestep range [0.0, 0.005]
Starting training on device: cpu
NFL LSTM model 1/201 completed
Split training data: 136 train, 16 validation

Training direct prediction LSTM model for timestep range [0.005, 0.01]
Starting training on device: cpu
NFL LSTM model 2/201 completed
Split training data: 279 train, 31 validation

Training direct prediction LSTM model for timestep range [0.01, 0.015]
Starting training on device: cpu
NFL LSTM model 3/201 completed
Split training data: 223 train, 25 validation

Training direct prediction LSTM model for timestep range [0.015, 0.02]
Starting training on device: cpu
NFL LSTM model 4/201 completed
Split training data: 257 train, 29 validation

Training direct prediction LSTM model for timestep range [0.02, 0.025]
Starting training on device: cpu
NFL LSTM model 5/201 completed
Split training data: 276 train, 31 validation

Training direct prediction LSTM model for ti

In [39]:
all_models["nn"] = setup_direct_models(training_data, None, num_models=201)

Original data shape: (477, 11)
Flattened data shape: (477, 11)
Split training data: 429 train, 48 validation

Training direct prediction model for timestep range [0.0, 0.005]
Starting training on device: cpu
Early stopping at epoch 82
Best epoch: 72, Train Acc: 0.8159, Train Loss: 0.1356, Val Acc: 0.7708, Val Loss: 0.1664
Restored model from best epoch 72 with val_loss: 0.166404
NFL direct model 1/201 completed
Original data shape: (180, 11)
Flattened data shape: (180, 11)
Split training data: 162 train, 18 validation

Training direct prediction model for timestep range [0.005, 0.01]
Starting training on device: cpu
Early stopping at epoch 23
Best epoch: 13, Train Acc: 0.6605, Train Loss: 0.2175, Val Acc: 0.6667, Val Loss: 0.2309
Restored model from best epoch 13 with val_loss: 0.230942
NFL direct model 2/201 completed
Original data shape: (300, 11)
Flattened data shape: (300, 11)
Split training data: 270 train, 30 validation

Training direct prediction model for timestep range [0.01, 

In [40]:
other_features = [
            "type.id",             # Play type (categorical)
            "home_has_possession", # Binary indicator
            "end.down",            # Down number (1-4, discrete)
            "home_timeouts_left",  # Discrete count (0-3)
            "away_timeouts_left",  # Discrete count (0-3)
        ]
numeric_features = [
    "score_difference",
    "relative_strength", 
    "end.yardsToEndzone", 
    "end.distance", 
    "field_position_shift"
]
all_models["logistic"] = setup_logistic_regression_models(training_data, None, numeric_features, other_features, features)


Processing timestep: 0.0
Timestep 0.00% : Training Loss = 0.6468, Accuracy = 0.6543, Test Loss = 0.5912, Test Accuracy = 0.6944
Processing timestep: 0.005
Timestep 0.50% : Training Loss = 0.6015, Accuracy = 0.6601, Test Loss = 0.6674, Test Accuracy = 0.6667
Processing timestep: 0.01
Timestep 1.00% : Training Loss = 0.6395, Accuracy = 0.6314, Test Loss = 0.6252, Test Accuracy = 0.6000
Processing timestep: 0.015
Timestep 1.50% : Training Loss = 0.6224, Accuracy = 0.6345, Test Loss = 0.7122, Test Accuracy = 0.5952
Processing timestep: 0.02
Timestep 2.00% : Training Loss = 0.6600, Accuracy = 0.6345, Test Loss = 0.6229, Test Accuracy = 0.6444
Processing timestep: 0.025
Timestep 2.50% : Training Loss = 0.6244, Accuracy = 0.6475, Test Loss = 0.7668, Test Accuracy = 0.6000
Processing timestep: 0.03
Timestep 3.00% : Training Loss = 0.6395, Accuracy = 0.6333, Test Loss = 0.6520, Test Accuracy = 0.5814
Processing timestep: 0.035
Timestep 3.50% : Training Loss = 0.6218, Accuracy = 0.6584, Test Los

In [41]:
all_models["xgboost"] = setup_xgboost_models(training_data, None, numeric_features, other_features, features)


Timestep 0.00%(Calibrated): Training Loss = 0.3277, Accuracy = 0.6723, Validation Loss = 0.1846, Validation Accuracy = 0.7333
Timestep 0.50%(Calibrated): Training Loss = 0.3333, Accuracy = 0.6667, Validation Loss = 0.2039, Validation Accuracy = 0.7111
Timestep 1.00%(Calibrated): Training Loss = 0.3644, Accuracy = 0.6356, Validation Loss = 0.2269, Validation Accuracy = 0.6400
Timestep 1.50%(Calibrated): Training Loss = 0.3619, Accuracy = 0.6381, Validation Loss = 0.2282, Validation Accuracy = 0.6143
Timestep 2.00%(Calibrated): Training Loss = 0.3955, Accuracy = 0.6045, Validation Loss = 0.2119, Validation Accuracy = 0.6622
Timestep 2.50%(Calibrated): Training Loss = 0.3862, Accuracy = 0.6138, Validation Loss = 0.1661, Validation Accuracy = 0.7439
Timestep 3.00%(Calibrated): Training Loss = 0.3255, Accuracy = 0.6745, Validation Loss = 0.2205, Validation Accuracy = 0.6338
Timestep 3.50%(Calibrated): Training Loss = 0.3508, Accuracy = 0.6492, Validation Loss = 0.1921, Validation Accuracy =

In [83]:
from collections import defaultdict
import numpy as np
print(all_models.keys())
all_models_order = ["xgboost", "nn", "logistic", "lstm"] # Strict ordering of models
def generate_ensemble_matrix(models, all_models_order, data_dict_seq):
    """Generate predictions from a specific model type on given data"""
    predictions = {}
    # predictions:
        # timestep:
            # "predictions": [model_1_predictions, model_2_predictions, ..., model_n_predictions, 
            # model_1_predictions_seq, model_2_predictions_seq, ..., model_n_predictions_seq],
            # "y_true": y_true
    for timestep in data_dict_seq:
        print(f"Generating predictions for timestep {timestep}")
        # For each entry, take the last array from the "rows" list and pair with its label
        non_sequential_data_for_timestep = [{"rows": entry["rows"][-1], "label": entry["label"]} for entry in data_dict_seq[timestep]]
        X = np.array([row["rows"] for row in non_sequential_data_for_timestep])
        y = np.array([row["label"] for row in non_sequential_data_for_timestep])
        X_seq = np.array([row["rows"] for row in data_dict_seq[timestep]])
        predictions[timestep] = {"predictions": [], "y_true": []}
        for i in range(len(X)):
            predictions[timestep]["predictions"].append(np.array([
                models[model][timestep].predict_proba(np.expand_dims(X_seq[i], axis=0))[:, 1].item() if model == "lstm" else models[model][timestep].predict_proba(np.expand_dims(X[i], axis=0))[:, 1].item()
                for model in all_models_order
            ]))
            predictions[timestep]["y_true"].append(y[i])
        predictions[timestep]["y_true"] = np.array(predictions[timestep]["y_true"])
        predictions[timestep]["predictions"] = np.array(predictions[timestep]["predictions"])
    return predictions

ensemble_matrices = generate_ensemble_matrix(all_models, all_models_order, ensemble_data_seq)


dict_keys(['lstm', 'nn', 'logistic', 'xgboost'])
Generating predictions for timestep 0.0
Generating predictions for timestep 0.005
Generating predictions for timestep 0.01
Generating predictions for timestep 0.015
Generating predictions for timestep 0.02
Generating predictions for timestep 0.025
Generating predictions for timestep 0.03
Generating predictions for timestep 0.035
Generating predictions for timestep 0.04
Generating predictions for timestep 0.045
Generating predictions for timestep 0.05
Generating predictions for timestep 0.055
Generating predictions for timestep 0.06
Generating predictions for timestep 0.065
Generating predictions for timestep 0.07
Generating predictions for timestep 0.075
Generating predictions for timestep 0.08
Generating predictions for timestep 0.085
Generating predictions for timestep 0.09
Generating predictions for timestep 0.095
Generating predictions for timestep 0.1
Generating predictions for timestep 0.105
Generating predictions for timestep 0.11

In [84]:
class LogisticRegressionMetaModel:
    def __init__(self):
        self.meta_model = LogisticRegression(random_state=42, max_iter=1000)

    def fit(self, X, y, X_val=None, y_val=None):
        self.meta_model.fit(X, y)
        if X_val is not None and y_val is not None:
            self.meta_model.fit(X_val, y_val)
    def predict(self, X):
        return 1 if self.meta_model.predict_proba(X)[:, 1] > 0.5 else 0
    def predict_proba(self, X):
        return self.meta_model.predict_proba(X)

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, ClassifierMixin

class SimpleMetaNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleMetaNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

class NeuralNetworkMetaModel(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim=None, lr=0.01, epochs=30, batch_size=32, device=None):
        self.input_dim = input_dim
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None

    def fit(self, X, y, X_val=None, y_val=None):
        if self.input_dim is None:
            self.input_dim = X.shape[1]
        self.model = SimpleMetaNN(self.input_dim).to(self.device)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)
        dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        self.model.train()
        for epoch in range(self.epochs):
            for xb, yb in loader:
                optimizer.zero_grad()
                preds = self.model(xb)
                loss = criterion(preds, yb)
                loss.backward()
                optimizer.step()
        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            probs = self.model(X_tensor).cpu().numpy().flatten()
        return (probs > 0.5).astype(int)

    def predict_proba(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            probs = self.model(X_tensor).cpu().numpy().flatten()
        return np.column_stack([1 - probs, probs])

In [161]:
def setup_meta_models(ensemble_matrices, all_models, all_models_order, strategy='meta_model', meta_model=None):
    models = {}
    for timestep in ensemble_matrices:
        print(f"Training meta-model for timestep {timestep}")
        ensemble_matrix = ensemble_matrices[timestep]
        x_train = ensemble_matrix["predictions"]
        y_train = ensemble_matrix["y_true"]
        all_models_for_timestep = {model_name: all_models[model_name][timestep] for model_name in all_models_order}
        models[timestep] = EnsemblePredictor(all_models_for_timestep, all_models_order, features, strategy, meta_model)
        models[timestep].train_ensemble(x_train, y_train, objective='brier')
    return models

ensemble_models = setup_meta_models(ensemble_matrices, all_models, all_models_order, strategy='weighted_average', meta_model=LogisticRegressionMetaModel())

Training meta-model for timestep 0.0
Training ensemble for this timestep...
Optimizing ensemble weights for this timestep...
  Optimized weights: {'xgboost': 0.0, 'nn': 0.1183, 'logistic': 0.4258, 'lstm': 0.4559} (score: 0.222985)
Training meta-model for timestep 0.005
Training ensemble for this timestep...
Optimizing ensemble weights for this timestep...
  Optimized weights: {'xgboost': 0.0, 'nn': 0.556, 'logistic': 0.2871, 'lstm': 0.157} (score: 0.245724)
Training meta-model for timestep 0.01
Training ensemble for this timestep...
Optimizing ensemble weights for this timestep...
  Optimized weights: {'xgboost': 0.2662, 'nn': 0.0, 'logistic': 0.1808, 'lstm': 0.5531} (score: 0.225142)
Training meta-model for timestep 0.015
Training ensemble for this timestep...
Optimizing ensemble weights for this timestep...
  Optimized weights: {'xgboost': 0.0711, 'nn': 0.2089, 'logistic': 0.0, 'lstm': 0.72} (score: 0.233980)
Training meta-model for timestep 0.02
Training ensemble for this timestep..

In [ ]:
# Test accuracy and Brier score of model for each timestep on test data and plot
accuracies = []
brier_scores = []
timesteps = []
def brier_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

for timestep, i in zip(ensemble_models, test_data.keys()):
    model = ensemble_models[timestep]
    # Convert test data to array
    y_test = np.array([row["label"] for row in test_data_seq[i]])
    X_test = np.array([row["rows"] for row in test_data_seq[i]])
    
    # Calculate accuracy
    accuracy = model.score(X_test, y_test)
    
    # Calculate Brier score
    y_test_pred_proba = model.predict_proba(X_test)[:, 1]  # Get probability predictions
    brier_score = brier_loss(y_test, y_test_pred_proba)
    
    print(f"Timestep {timestep:.2%}: Accuracy = {accuracy:.4f}, Brier Score = {brier_score:.4f}")
    accuracies.append(accuracy)
    brier_scores.append(brier_score)
    timesteps.append(timestep)

# Create subplots for both metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot accuracy
ax1.plot(timesteps, accuracies, 'b-', linewidth=2, marker='o', markersize=3)
ax1.set_xlabel("Timestep")
ax1.set_ylabel("Accuracy")
ax1.set_title("Test Accuracy vs Timestep")
ax1.grid(True, alpha=0.3)
ax1.set_ylim([min(accuracies) * 0.95, max(accuracies) * 1.05])

# Plot Brier score
ax2.plot(timesteps, brier_scores, 'r-', linewidth=2, marker='s', markersize=3)
ax2.set_xlabel("Timestep")
ax2.set_ylabel("Brier Score")
ax2.set_title("Test Brier Score vs Timestep")
ax2.grid(True, alpha=0.3)
ax2.set_ylim([min(brier_scores) * 0.95, max(brier_scores) * 1.05])

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nSummary Statistics:")
print(f"Average Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Average Brier Score: {np.mean(brier_scores):.4f} ± {np.std(brier_scores):.4f}")
print(f"Best Accuracy: {max(accuracies):.4f} at timestep {timesteps[np.argmax(accuracies)]:.2%}")
print(f"Best Brier Score: {min(brier_scores):.4f} at timestep {timesteps[np.argmin(brier_scores)]:.2%}")

In [146]:
model = ensemble_models[0]
model.predict_proba(np.arange(110).reshape(2, 5, 11))

Model name:  xgboost
(2, 5, 11)
(2, 11)
[[0.         1.        ]
 [0.19230769 0.80769231]]
Model name:  nn
(2, 5, 11)
(2, 11)
[[1. 0.]
 [1. 0.]]
Model name:  logistic
(2, 5, 11)
(2, 11)
[[0. 1.]
 [0. 1.]]
Model name:  lstm
(2, 5, 11)
[[0.79934996 0.20065005]
 [0.7205218  0.27947822]]


array([[0.48275698, 0.51724302],
       [0.44681822, 0.55318178]])

In [ ]:
# Write predictions to csv file
from process_data import write_predictions
write_predictions(ensemble_models, interpolated_dir, [2024], 4, features, replace_nan_val = 0, phat_b = "ensemble_phat_b_model")

Loading data for 2024
Processed file:  game_401671629.csv
Processed file:  game_401671601.csv
Processed file:  game_401671826.csv
Processed file:  game_401671832.csv
Processed file:  game_401671749.csv
Processed file:  game_401671775.csv
Processed file:  game_401671761.csv
Processed file:  game_401671760.csv
Processed file:  game_401671774.csv
Processed file:  game_401671748.csv
Processed file:  game_401671833.csv
Processed file:  game_401671827.csv
Processed file:  game_401671600.csv
Processed file:  game_401671628.csv
Processed file:  game_401671616.csv
Processed file:  game_401671831.csv
Processed file:  game_401671825.csv
Processed file:  game_401671819.csv
Processed file:  game_401671762.csv
Processed file:  game_401671776.csv
Processed file:  game_401671789.csv
Processed file:  game_401671788.csv
Processed file:  game_401671777.csv
Processed file:  game_401671763.csv
Processed file:  game_401671818.csv
Processed file:  game_401671824.csv
Processed file:  game_401671830.csv
Proces